In [ ]:
!pip -q install torch scikit-learn numpy openai

import time, random, json, os, getpass
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
from openai import OpenAI

SEED = 7
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

DEVICE = torch.device("cpu")
print("Device:", DEVICE)

In [ ]:
X, y = make_classification(
    n_samples=60000,
    n_features=30,
    n_informative=18,
    n_redundant=8,
    weights=[0.985, 0.015],
    class_sep=1.5,
    flip_y=0.01,
    random_state=SEED
)

X = X.astype(np.float32)
y = y.astype(np.int64)

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

server_scaler = StandardScaler()
X_train_full_s = server_scaler.fit_transform(X_train_full).astype(np.float32)
X_test_s = server_scaler.transform(X_test).astype(np.float32)

test_loader = DataLoader(
    TensorDataset(torch.from_numpy(X_test_s), torch.from_numpy(y_test)),
    batch_size=1024,
    shuffle=False
)

In [ ]:
def dirichlet_partition(y, n_clients=10, alpha=0.35):
    classes = np.unique(y)
    idx_by_class = [np.where(y == c)[0] for c in classes]
    client_idxs = [[] for _ in range(n_clients)]
    for idxs in idx_by_class:
        np.random.shuffle(idxs)
        props = np.random.dirichlet(alpha * np.ones(n_clients))
        cuts = (np.cumsum(props) * len(idxs)).astype(int)
        prev = 0
        for cid, cut in enumerate(cuts):
            client_idxs[cid].extend(idxs[prev:cut].tolist())
            prev = cut
    return [np.array(ci, dtype=np.int64) for ci in client_idxs]

NUM_CLIENTS = 10
client_idxs = dirichlet_partition(y_train_full, NUM_CLIENTS, 0.35)

def make_client_split(X, y, idxs):
    Xi, yi = X[idxs], y[idxs]
    if len(np.unique(yi)) < 2:
        other = np.where(y == (1 - yi[0]))[0]
        add = np.random.choice(other, size=min(10, len(other)), replace=False)
        Xi = np.concatenate([Xi, X[add]])
        yi = np.concatenate([yi, y[add]])
    return train_test_split(Xi, yi, test_size=0.15, stratify=yi, random_state=SEED)

client_data = [make_client_split(X_train_full, y_train_full, client_idxs[c]) for c in range(NUM_CLIENTS)]

def make_client_loaders(Xtr, ytr, Xva, yva):
    sc = StandardScaler()
    Xtr_s = sc.fit_transform(Xtr).astype(np.float32)
    Xva_s = sc.transform(Xva).astype(np.float32)
    tr = DataLoader(TensorDataset(torch.from_numpy(Xtr_s), torch.from_numpy(ytr)), batch_size=512, shuffle=True)
    va = DataLoader(TensorDataset(torch.from_numpy(Xva_s), torch.from_numpy(yva)), batch_size=512)
    return tr, va

client_loaders = [make_client_loaders(*cd) for cd in client_data]

In [ ]:
class FraudNet(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x).squeeze(-1)

def get_weights(model):
    return [p.detach().cpu().numpy() for p in model.state_dict().values()]

def set_weights(model, weights):
    keys = list(model.state_dict().keys())
    model.load_state_dict({k: torch.tensor(w) for k, w in zip(keys, weights)}, strict=True)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    bce = nn.BCEWithLogitsLoss()
    ys, ps, losses = [], [], []
    for xb, yb in loader:
        logits = model(xb)
        losses.append(bce(logits, yb.float()).item())
        ys.append(yb.numpy())
        ps.append(torch.sigmoid(logits).numpy())
    y_true = np.concatenate(ys)
    y_prob = np.concatenate(ps)
    return {
        "loss": float(np.mean(losses)),
        "auc": roc_auc_score(y_true, y_prob),
        "ap": average_precision_score(y_true, y_prob),
        "acc": accuracy_score(y_true, (y_prob >= 0.5).astype(int))
    }

def train_local(model, loader, lr):
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    bce = nn.BCEWithLogitsLoss()
    model.train()
    for xb, yb in loader:
        opt.zero_grad()
        loss = bce(model(xb), yb.float())
        loss.backward()
        opt.step()

In [ ]:
def fedavg(weights, sizes):
    total = sum(sizes)
    return [
        sum(w[i] * (s / total) for w, s in zip(weights, sizes))
        for i in range(len(weights[0]))
    ]

ROUNDS = 10
LR = 5e-4

global_model = FraudNet(X_train_full.shape[1])
global_weights = get_weights(global_model)

for r in range(1, ROUNDS + 1):
    client_weights, client_sizes = [], []
    for cid in range(NUM_CLIENTS):
        local = FraudNet(X_train_full.shape[1])
        set_weights(local, global_weights)
        train_local(local, client_loaders[cid][0], LR)
        client_weights.append(get_weights(local))
        client_sizes.append(len(client_loaders[cid][0].dataset))
    global_weights = fedavg(client_weights, client_sizes)
    set_weights(global_model, global_weights)
    metrics = evaluate(global_model, test_loader)
    print(f"Round {r}: {metrics}")

In [15]:
OPENAI_API_KEY = getpass.getpass("Enter OPENAI_API_KEY (input hidden): ").strip()

if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    client = OpenAI()

    summary = {
        "rounds": ROUNDS,
        "num_clients": NUM_CLIENTS,
        "final_metrics": metrics,
        "client_sizes": [len(client_loaders[c][0].dataset) for c in range(NUM_CLIENTS)],
        "client_fraud_rates": [float(client_data[c][1].mean()) for c in range(NUM_CLIENTS)]
    }

    prompt = (
        "Write a concise internal fraud-risk report.\n"
        "Include executive summary, metric interpretation, risks, and next steps.\n\n"
        + json.dumps(summary, indent=2)
    )

    resp = client.responses.create(model="gpt-5.2", input=prompt)
    print(resp.output_text)

Device: cpu


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Client sizes: [4157, 3254, 3823, 6567, 6137, 361, 4943, 881, 8794, 1893]
Client fraud rates: [0.008900649506855906, 0.002458512599877074, 0.020664399686110384, 0.0012182122734886553, 0.0008147303242626691, 0.2548476454293629, 0.11673073032571313, 0.004540295119182747, 0.00045485558335228563, 0.002641310089804543]


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


[Round 01] test_loss=0.6095 AUC=0.2353 AP=0.0118 Acc=0.9767


...

[Round 09] test_loss=0.1139 AUC=0.4557 AP=0.0180 Acc=0.9802


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


[Round 10] test_loss=0.1044 AUC=0.4962 AP=0.0203 Acc=0.9802
Done in 15.8s


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Enter OPENAI_API_KEY (input hidden): ··········

=== OpenAI Risk Report ===

1) **Executive summary**
- The FedAvg run over 10 banks (10 rounds, 1 local epoch, lr=5e-4) produced **near-random discrimination** on the global test set (**AUC 0.496**, **AP 0.020**), despite high overall accuracy (**0.980**).
- Client label distributions are **extremely non-IID**: fraud rates range from **0.045% to 25.5%** (≈560× spread), implying strong gradient conflict and a high risk that a single global model underperforms for many banks.
- The reported **0.980 accuracy is not meaningful** in this imbalanced setting; it is consistent with predicting “non-fraud” most of the time.
- While FL avoids centralizing raw transaction data, it **does not guarantee perfect privacy**; model updates can still leak information without additional protections.

2) **Metrics interpretation (AUC/AP/Acc)**
AUC ~0.50 indicates the model is essentially unable to rank fraud above non-fraud on the global test distribution.  